In [ ]:
import requests
from lxml import etree
import json
import os
from selenium import webdriver
import pandas as pd
# set default encoding
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [ ]:
# heads.txt 从浏览器复制
with open("heads.txt") as f:
    txt = f.readlines()
headers = {str(i.split(": ")[0]):str(i.split(": ")[1].strip('\n').strip('\r')) for i in txt}

In [ ]:
url = "https://www.lagou.com/jobs/positionAjax.json?city=%E4%B8%8A%E6%B5%B7"
kd = u'数据挖掘'
data = {
    'first': False,
    'pn': 1,
    'kd': kd,
}

In [ ]:
try:
    r = requests.post(url=url,headers=headers, data=data)
    r.raise_for_status()
except Exception as e:
    print "Failed to open url:{}".format(e)

In [ ]:
total_count = r.json()["content"]["positionResult"]["totalCount"]
total_page = min(total_count/15 + 1, 30)
all_result = []
browser = webdriver.Chrome()
for page in range(total_page):
    if page == 0:
        data = {'first': True, 'pn':1, 'kd': kd,}
    else:
        data = {'first': False, 'pn':page+1, 'kd': kd,}
    r = requests.post(url=url,headers=headers, data=data)
    #
    tabel = r.json()
    for pos_no in range(15):
        item = {}
        inf = tabel['content']['positionResult']['result'][pos_no]
        position_id = inf['positionId']
        page_url = "https://www.lagou.com/jobs/{}.html".format(position_id)
        browser.get(page_url)
        item['des'] = browser.find_element_by_xpath("//dl[@id='job_detail']/dd[@class='job_bt']").text
        item['size'] = inf['companySize']
        item['edu'] = inf['education']
        item['salary'] = inf['salary']
        item['labels'] = inf['companyLabelList']
        item['name'] = inf['companyFullName']
        item['work_year'] = inf['workYear']
        item['advantage'] = inf['positionAdvantage']
        item['finance_stage'] = inf['financeStage']
        item['district'] = inf['district']
        item['industry_field'] = inf['industryField']
        item['position_id'] = position_id
        all_result.append(item)

In [ ]:
result = pd.DataFrame(all_result)
result.to_csv("data_analysis.csv")